In [ ]:
import sys
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd.variable import *
import torch.optim as optim
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib import rc
import setGPU
from sklearn.metrics import roc_curve, auc
import h5py

rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
rcParams['font.size'] = 22
rcParams['text.latex.preamble'] = [
#       r'\usepackage{siunitx}',   # i need upright \micro symbols, but you need...
#       r'\sisetup{detect-all}',   # ...this to force siunitx to actually use your fonts
       r'\usepackage{helvet}',    # set the normal font here
       r'\usepackage{sansmath}',  # load up the sansmath so that math -> helvet
       r'\sansmath'               # <- tricky! -- gotta actually tell tex to use!
]  
rc('text', usetex=True)

In [2]:
save_path = '/nfshome/emoreno/IN/data/opendata/test/'
#test_0 = np.load(save_path + 'test_features_0.npy')
#test_0 = np.swapaxes(test_0, 1, 2)
#training_0 = np.load(save_path + 'train_withSpectator_features_0.npy') #per jet constituents
#training_0 = np.swapaxes(training_0, 1, 2)
#training_2 = np.load(save_path + 'train_features_2.npy') #30 features of 60 charged particles
#training_2 = np.swapaxes(training_2, 1, 2)
#training_3 = np.load(save_path + 'train_features_3.npy') #14 features of 5 secondary vertices
#training_3 = np.swapaxes(training_3, 1, 2)
#target = np.load(save_path + 'train_truth_0.npy')
#test_0 = np.load(save_path + 'test_0.npy')
#test_0 = np.swapaxes(test_0, 1, 2)
#print(test_0.shape)
#test_1 = np.load(save_path + 'test_1.npy')
#test_1 = np.swapaxes(test_1, 1, 2)

# Load tracks dataset
test_2 = np.load(save_path + 'test_2.npy')
test_2 = np.swapaxes(test_2, 1, 2)

# Load SV dataset
test_3 = np.load(save_path + 'test_3.npy')
test_3 = np.swapaxes(test_3, 1, 2)
target_test = np.load(save_path + 'truth_0.npy')

params_0 = ['fj_jetNTracks',
          'fj_nSV',
          'fj_tau0_trackEtaRel_0',
          'fj_tau0_trackEtaRel_1',
          'fj_tau0_trackEtaRel_2',
          'fj_tau1_trackEtaRel_0',
          'fj_tau1_trackEtaRel_1',
          'fj_tau1_trackEtaRel_2',
          'fj_tau_flightDistance2dSig_0',
          'fj_tau_flightDistance2dSig_1',
          'fj_tau_vertexDeltaR_0',
          'fj_tau_vertexEnergyRatio_0',
          'fj_tau_vertexEnergyRatio_1',
          'fj_tau_vertexMass_0',
          'fj_tau_vertexMass_1',
          'fj_trackSip2dSigAboveBottom_0',
          'fj_trackSip2dSigAboveBottom_1',
          'fj_trackSip2dSigAboveCharm_0',
          'fj_trackSipdSig_0',
          'fj_trackSipdSig_0_0',
          'fj_trackSipdSig_0_1',
          'fj_trackSipdSig_1',
          'fj_trackSipdSig_1_0',
          'fj_trackSipdSig_1_1',
          'fj_trackSipdSig_2',
          'fj_trackSipdSig_3',
          'fj_z_ratio'
          ]

params_1 = ['pfcand_ptrel',
          'pfcand_erel',
          'pfcand_phirel',
          'pfcand_etarel',
          'pfcand_deltaR',
          'pfcand_puppiw',
          'pfcand_drminsv',
          'pfcand_drsubjet1',
          'pfcand_drsubjet2',
          'pfcand_hcalFrac'
         ]

params_2 = ['track_ptrel',     
          'track_erel',     
          'track_phirel',     
          'track_etarel',     
          'track_deltaR',
          'track_drminsv',     
          'track_drsubjet1',     
          'track_drsubjet2',
          'track_dz',     
          'track_dzsig',     
          'track_dxy',     
          'track_dxysig',     
          'track_normchi2',     
          'track_quality',     
          'track_dptdpt',     
          'track_detadeta',     
          'track_dphidphi',     
          'track_dxydxy',     
          'track_dzdz',     
          'track_dxydz',     
          'track_dphidxy',     
          'track_dlambdadz',     
          'trackBTag_EtaRel',     
          'trackBTag_PtRatio',     
          'trackBTag_PParRatio',     
          'trackBTag_Sip2dVal',     
          'trackBTag_Sip2dSig',     
          'trackBTag_Sip3dVal',     
          'trackBTag_Sip3dSig',     
          'trackBTag_JetDistVal'
         ]

params_3 = ['sv_ptrel',
          'sv_erel',
          'sv_phirel',
          'sv_etarel',
          'sv_deltaR',
          'sv_pt',
          'sv_mass',
          'sv_ntracks',
          'sv_normchi2',
          'sv_dxy',
          'sv_dxysig',
          'sv_d3d',
          'sv_d3dsig',
          'sv_costhetasvpv'
         ]

test_sv = test_3
test = test_2
N = test.shape[2]
params = params_2
params_sv = params_3

In [3]:
import itertools
from sklearn import utils
use_cuda = True
device = torch.device("cuda" if use_cuda else "cpu")

class GraphNet(nn.Module):
    def __init__(self, n_constituents, n_targets, params, hidden):
        super(GraphNet, self).__init__()
        self.hidden = hidden
        self.P = len(params)
        self.N = n_constituents
        self.S = test_sv.shape[1]
        self.Nv = test_sv.shape[2]
        self.Nr = self.N * (self.N - 1)
        self.Dr = 0
        self.De = 5
        self.Dx = 0
        self.Do = 6
        self.n_targets = n_targets
        self.assign_matrices()
        self.assign_matrices_SV()
        #self.switch = switch
        
        self.Ra = torch.ones(self.Dr, self.Nr)
        self.fr1 = nn.Linear(2 * self.P + self.Dr, hidden).cuda()
        self.fr1_sv = nn.Linear(self.S + self.P + self.Dr, hidden).cuda()
        self.fr2 = nn.Linear(hidden, hidden/2).cuda()
        self.fr3 = nn.Linear(hidden/2, self.De).cuda()
        self.fo1 = nn.Linear(self.P + self.Dx + (2 * self.De), hidden).cuda()
        self.fo2 = nn.Linear(hidden, hidden/2).cuda()
        self.fo3 = nn.Linear(hidden/2, self.Do).cuda()
        self.fc1 = nn.Linear(self.Do * self.N, hidden).cuda()
        self.fc2 = nn.Linear(hidden, hidden/2).cuda()
        self.fc3 = nn.Linear(hidden/2, self.n_targets).cuda()
        self.fc_fixed = nn.Linear(self.Do, self.n_targets).cuda()
        #self.gru = nn.GRU(input_size = self.Do, hidden_size = 20, bidirectional = False).cuda()
            
    def assign_matrices(self):
        self.Rr = torch.zeros(self.N, self.Nr)
        self.Rs = torch.zeros(self.N, self.Nr)
        receiver_sender_list = [i for i in itertools.product(range(self.N), range(self.N)) if i[0]!=i[1]]
        for i, (r, s) in enumerate(receiver_sender_list):
            self.Rr[r, i] = 1
            self.Rs[s, i] = 1
        self.Rr = (self.Rr).cuda()
        self.Rs = (self.Rs).cuda()
    
    def assign_matrices_SV(self):
        self.Rk = torch.zeros(self.N, self.Nr)
        self.Rv = torch.zeros(self.Nv, self.Nr)
        receiver_sender_list = [i for i in itertools.product(range(self.N), range(self.Nv)) if i[0]!=i[1]]
        for i, (k, v) in enumerate(receiver_sender_list):
            self.Rk[k, i] = 1
            self.Rv[v, i] = 1
        self.Rk = (self.Rk).cuda()
        self.Rv = (self.Rv).cuda()
        
    def forward(self, x, y):
        ###PF Candidate - PF Candidate###
        Orr = self.tmul(x, self.Rr)
        Ors = self.tmul(x, self.Rs)
        B = torch.cat([Orr, Ors], 1)
        ### First MLP ###
        B = torch.transpose(B, 1, 2).contiguous()
        B = nn.functional.relu(self.fr1(B.view(-1, 2 * self.P + self.Dr)))
        B = nn.functional.relu(self.fr2(B))
        E = nn.functional.relu(self.fr3(B).view(-1, self.Nr, self.De))
        del B
        E = torch.transpose(E, 1, 2).contiguous()
        Ebar = self.tmul(E, torch.transpose(self.Rr, 0, 1).contiguous())
        del E
        
        ####Secondary Vertex - PF Candidate### 
        Ork = self.tmul(x, self.Rk)
        Orv = self.tmul(y, self.Rv)
        B = torch.cat([Ork, Orv], 1)
        ### First MLP ###
        B = torch.transpose(B, 1, 2).contiguous()
        B = nn.functional.relu(self.fr1_sv(B.view(-1, self.S + self.P + self.Dr)))
        B = nn.functional.relu(self.fr2(B))
        E = nn.functional.relu(self.fr3(B).view(-1, self.Nr, self.De))
        del B
        E = torch.transpose(E, 1, 2).contiguous()
        Ebar_sv = self.tmul(E, torch.transpose(self.Rr, 0, 1).contiguous())
        del E

        ####Final output matrix###
        C = torch.cat([x, Ebar], 1)
        del Ebar
        C = torch.cat([C, Ebar_sv], 1)
        del Ebar_sv
        C = torch.transpose(C, 1, 2).contiguous()
        ### Second MLP ###
        C = nn.functional.relu(self.fo1(C.view(-1, self.P + self.Dx + (2 * self.De))))
        C = nn.functional.relu(self.fo2(C))
        O = nn.functional.relu(self.fo3(C).view(-1, self.N, self.Do))
        #Taking the mean/sum of each column
        #N = torch.mean(O, dim=1)
        N = torch.sum(O, dim=1)
        del C
        ### Classification MLP ###
        #N = nn.functional.relu(self.fc1(O.view(-1, self.Do * self.N)))
        del O
        #N = nn.functional.relu(self.fc2(N))
        #N = nn.functional.relu(self.fc3(N))
        N = nn.functional.relu(self.fc_fixed(N))
        #P = np.array(N.data.cpu().numpy())
        #N = np.zeros((128, 1, 6))
        #for i in range(batch_size):
        #    N[i] = np.array(np.split(P[i], self.Do))
        #    N[1] = [P[i]]
        #N, hn = self.gru(torch.tensor(N).cuda())
        #print((N).shape)
        return N 
            
    def tmul(self, x, y):  #Takes (I * J * K)(K * L) -> I * J * L 
        x_shape = x.size()
        y_shape = y.size()
        return torch.mm(x.view(-1, x_shape[2]), y).view(-1, x_shape[1], y_shape[1])

#n_targets = test.shape[1]
n_targets = 2
N = 60
gnn = GraphNet(N, n_targets, params, 15)
#gnn.load_state_dict(torch.load('IN_opendata_V2'))

def get_sample(training1, training2, target, choice):
    target_vals = np.argmax(target, axis = 1)
    ind, = np.where(target_vals == choice)
    chosen_ind = np.random.choice(ind, 300000)
    return training1[chosen_ind], training2[chosen_ind], target[chosen_ind]

def get_sample_train(training1, training2, target, choice):
    target_vals = np.argmax(target, axis = 1)
    ind, = np.where(target_vals == choice)
    chosen_ind = ind
    #chosen_ind = np.random.choice(ind, 200000)
    return training1[chosen_ind], training2[chosen_ind], target[chosen_ind]


In [5]:
# Load predictions on test dataset 
prediction = np.load('/nfshome/emoreno/IN/avikar-surf2017/opendata/IN_out_Run2.npy')

In [10]:
# Create ROC Plot
from sklearn.metrics import roc_curve, roc_auc_score
softmax = torch.nn.Softmax(dim=1)

fpr, tpr, thresholds = roc_curve(target_test, prediction)
auc = roc_auc_score(target_test, prediction)

f, ax = plt.subplots(figsize=(10, 10))
ax.set_xlim(0,1)
ax.set_ylim(0.001,1)
sig=["Hbb"]
bkg=["QCD"]
eraText=r'2016 (13 TeV)'
xlab = '{} \\rightarrow {}'.format(sig[0][0], sig[0][-2]+'\\bar{'+sig[0][-1]+'}') 
ax.set_xlabel(r'Tagging efficiency ($\mathrm{}$)'.format('{'+xlab+'}'), ha='right', x=1.0)
ylab = ['{} \\rightarrow {}'.format(l[0], l[-2]+'\\bar{'+l[-1]+'}') if l[0][0] in ["H", "Z", "g"] else l for l in bkg ]
ax.set_ylabel(r'Mistagging rate ($\mathrm{}$)'.format("{"+", ".join(ylab)+"}"), ha='right', y=1.0)
leg = ax.legend(borderpad=1, frameon=False, loc=2, fontsize=16,
            title = ""+str(int(round((min(frame.fj_pt)))))+" $\mathrm{<\ jet\ p_T\ <}$ "+str(int(round((max(frame.fj_pt)))))+" GeV" \
          + "\n "+str(int(round((min(frame.fj_sdmass)))))+" $\mathrm{<\ jet\ m_{sd}\ <}$ "+str(int(round((max(frame.fj_sdmass)))))+" GeV"
                       )
#fpr_DeepDoubleB = np.load('fpr_DDB_opendata.npy')
#tpr_DeepDoubleB = np.load('tpr_DDB_opendata.npy')
#plt.figure(figsize=(12,10), dpi = 200)
lw = 2
ax.plot(tpr, fpr, color='darkorange',
         lw=lw, label='Interaction Network (area = %0.3f)' % auc)
#ax.plot(tpr_DeepDoubleB, fpr_DeepDoubleB, color='navy', lw=lw, label='DeepDoubleB (area = 0.955)')
ax.xaxis.set_major_locator(plticker.MultipleLocator(base=0.1))
ax.xaxis.set_minor_locator(plticker.MultipleLocator(base=0.02))
ax.tick_params(direction='in', axis='both', which='major', labelsize=15, length=12 )
ax.tick_params(direction='in', axis='both', which='minor' , length=6)
ax.xaxis.set_ticks_position('both')
ax.yaxis.set_ticks_position('both')    
ax.semilogy()
ax.grid(which='minor', alpha=0.5, axis='y', linestyle='dotted')
ax.grid(which='major', alpha=0.9, linestyle='dotted')
ax.annotate(eraText, xy=(0.80, 1.1), fontname='Helvetica', ha='left',
            bbox={'facecolor':'white', 'edgecolor':'white', 'alpha':0, 'pad':13}, annotation_clip=False)
ax.annotate('$\mathbf{CMS}$', xy=(0.01, 1.1), fontname='Helvetica', fontsize=24, fontweight='bold', ha='left',
            bbox={'facecolor':'white', 'edgecolor':'white', 'alpha':0, 'pad':13}, annotation_clip=False)
ax.annotate('$Simulation\ Preliminary$', xy=(0.115, 1.1), fontsize=18, fontstyle='italic', ha='left',
            annotation_clip=False)

ax.legend(loc="upper left")
f.show()
f.savefig('ROC_curve_opendata_2.pdf')

ValueError: multilabel-indicator format is not supported

In [ ]:
# Create Pileup vs Efficiency Plot
import bisect
from sklearn.metrics import roc_curve, roc_auc_score
import itertools
from sklearn import utils

NBINS= 10 # number of bins for loss function
MMAX = 40. # max value
MMIN = 0. # min value
binWidth = (MMAX - MMIN) / NBINS
datapoints = 250000
OHE_pv = []
auc_array = []
cuts = [0.05, 0.1, 0.15, 0.25, 0.5]
pileup = [[],[],[],[],[]]

for i in range(NBINS): 
    OHE_pv.append([])
    
# One hot encode data according to pileup bins
for i in range(datapoints):
    if np.sum(sum(test_3[i][7])) >= MMAX: 
        OHE_pv[-1].append(i)
    elif np.sum(sum(test_3[i][7])) <= MMIN: 
        OHE_pv[0].append(i)
    else:
        OHE_pv[int((np.sum(test_3[i][7]) - MMIN)/binWidth)].append(i)

use_cuda = True
device = torch.device("cuda" if use_cuda else "cpu")
softmax = torch.nn.Softmax(dim=1)
n_targets = target_test.shape[1]

def get_sample(training1, training2, target, choice):
    target_vals = np.argmax(target, axis = 1)
    ind, = np.where(target_vals == choice)
    chosen_ind = np.random.choice(ind, datapoints)
    return training1[chosen_ind], training2[chosen_ind], target[chosen_ind]

for i in range(NBINS): 
    training_temp = test[OHE_pv[i]]
    training_sv_temp = test_sv[OHE_pv[i]]
    target_temp = target_test[OHE_pv[i]]
    
    samples = [get_sample(training_temp, training_sv_temp, target_temp, i) for i in range(n_targets)]
    trainings = [i[0] for i in samples]
    trainings_sv = [i[1] for i in samples]
    targets = [i[2] for i in samples]
    
    print(np.array(trainings).shape)
    big_training = np.concatenate(trainings)
    big_training_sv = np.concatenate(trainings_sv)
    big_target = np.concatenate(targets)
    big_training, big_training_sv, big_target = utils.shuffle(big_training, big_training_sv, big_target)

    val_split = 0.1
    batch_size =128
    n_epochs = 250
    print(big_training[0][0][0])
    trainingv = (torch.FloatTensor(big_training)).cuda()
    trainingv_sv = (torch.FloatTensor(big_training_sv)).cuda()
    targetv = (torch.from_numpy(np.argmax(big_target, axis = 1)).long()).cuda()
    trainingv, valv = torch.split(trainingv, int(trainingv.size()[0] * (1 - val_split)))
    trainingv_sv, valv_sv = torch.split(trainingv_sv, int(trainingv_sv.size()[0] * (1 - val_split)))
    targetv, val_targetv = torch.split(targetv, int(targetv.size()[0] * (1 - val_split)))
    samples_random = np.random.choice(range(len(trainingv)), valv.size()[0]/100)
    out = np.array([])
    prediction = np.array([])
    for j in range(0, trainingv.size()[0], batch_size):
        out_test = softmax(gnn(trainingv[j:j + batch_size].cuda(), trainingv_sv[j:j + batch_size].cuda()))
        out_test = out_test.cpu().data.numpy()
        #print(out_test)
        for i in range(len(out_test)):
            if (out_test[i][0] > out_test[i][1]): 
                prediction = np.append(prediction, out_test[i][0])
                out = np.append(out, 0)
            else: 
                prediction = np.append(prediction, out_test[i][1])
                out = np.append(out, 1)

    for i in range(prediction.size): 
        if out[i] == 0: 
            prediction[i] = 1.0 - prediction[i]
            
    fpr, tpr, thresholds = roc_curve(targetv.cpu().data.numpy(), prediction)
    auc_array.append(roc_auc_score(targetv.cpu().data.numpy(), prediction))
    entry = 0
    #print(thresholds)
    
    events_above_min_bound = []
    for cut in cuts: 
        entry = bisect.bisect(fpr, cut)
        min_bound = thresholds[entry]
        count = 0
        for n in range(prediction.shape[0]):
            if prediction[n] > min_bound:
                count += 1
        events_above_min_bound.append(count)
    for p in range(len(cuts)):
        print(prediction.shape[0])
        print(events_above_min_bound[p])
        pileup[p].append((1./prediction.shape[0])* events_above_min_bound[p])


bins = []
for i in range(len(pileup[0])):
    bins.append(MMIN + i * binWidth)

f, ax = plt.subplots(figsize=(10, 10))
ax.set_xlim(0,50)
ax.set_ylim(0,1)
sig=["Hbb"]
bkg=["QCD"]
eraText=r'2016 (13 TeV)'
ylab = '{} \\rightarrow {}'.format(sig[0][0], sig[0][-2]+'\\bar{'+sig[0][-1]+'}') 
ax.set_ylabel(r'Tagging efficiency ($\mathrm{}$)'.format('{'+ylab+'}'), ha='right', x=1.0)
ax.set_ylabel(r'Number of Event Vertices (Pileup)', ha='right', y=1.0)
#plt.figure(figsize=(12,10), dpi = 200)
lw = 2
ax.plot(bins, pileup[0], label = '1% Mistag')
ax.plot(bins, pileup[1], label = '5% Mistag')
ax.plot(bins, pileup[2], label = '10% Mistag')
ax.plot(bins, pileup[3], label = '25% Mistag')
ax.plot(bins, pileup[4], label = '50% Mistag')
ax.xaxis.set_major_locator(plticker.MultipleLocator(base=0.1))
ax.xaxis.set_minor_locator(plticker.MultipleLocator(base=0.02))
ax.tick_params(direction='in', axis='both', which='major', labelsize=15, length=12 )
ax.tick_params(direction='in', axis='both', which='minor' , length=6)
#ax.xaxis.set_ticks_position('both')
#x.yaxis.set_ticks_position('both')    
#ax.semilogy()
ax.grid(which='minor', alpha=0.5, axis='y', linestyle='dotted')
ax.grid(which='major', alpha=0.9, linestyle='dotted')
ax.annotate(eraText, xy=(0.80, 1.1), fontname='Helvetica', ha='left',
            bbox={'facecolor':'white', 'edgecolor':'white', 'alpha':0, 'pad':13}, annotation_clip=False)
ax.annotate('$\mathbf{CMS}$', xy=(0.01, 1.1), fontname='Helvetica', fontsize=24, fontweight='bold', ha='left',
            bbox={'facecolor':'white', 'edgecolor':'white', 'alpha':0, 'pad':13}, annotation_clip=False)
ax.annotate('$Simulation\ Opendata$', xy=(0.115, 1.1), fontsize=18, fontstyle='italic', ha='left',
            annotation_clip=False)    
ax.show()    

plt.savefig('pileup_vs_efficiency_opendata')